In [ ]:
import pandas as pd
from sklearn import tree
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import random
import numpy as np

### Data Preparation and Clean up

In [2]:
file = "US_births(2018).csv"

In [3]:
# Read our Data file with the pandas library

file_df = pd.read_csv(file, encoding="ISO-8859-1")

C:\Users\Mercy\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
file_df.head()
#file_df.shape

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,4,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [5]:
file_df = file_df.dropna()

In [6]:
# Limit data to known recorded delivery methods only 
delivery_method = file_df[file_df['RDMETH_REC'].isin(['1','2','3','4','5','6']) ]
delivery_method.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,4,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [7]:
# print number of rows and columns in dataset
delivery_method.shape

(3799906, 55)

In [8]:
# Update delivery method to have only Cesareans and Vaginal birth
# delivery_method = delivery_method["RDMETH_REC"].replace({2:1, 4:3, 5:1, 6:3}, inplace=True)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 2),1,delivery_method.RDMETH_REC)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 4),3,delivery_method.RDMETH_REC)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 5),1,delivery_method.RDMETH_REC)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 6),3,delivery_method.RDMETH_REC)
delivery_method.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,3,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [9]:
# Limit the columns to only mother related columns. Eliminate father and baby columns
delivery_method = delivery_method[['RDMETH_REC','ATTEND','CIG_0','BMI','DMAR','LD_INDL','MAGER','M_Ht_In','NO_INFEC','NO_RISKS'
                                    ,'NO_MMORB','PWgt_R','WTGAIN','RF_CESAR','RF_CESARN','PRIORDEAD','PRIORLIVE','PRIORTERM']]
delivery_method.head()

,RDMETH_REC,ATTEND,CIG_0,BMI,DMAR,LD_INDL,MAGER,M_Ht_In,NO_INFEC,NO_RISKS,NO_MMORB,PWgt_R,WTGAIN,RF_CESAR,RF_CESARN,PRIORDEAD,PRIORLIVE,PRIORTERM
0,1,1,0,30.7,1,N,30,66,1,1,1,190,41,N,0,0,1,2
1,3,1,2,33.3,2,N,35,63,1,0,1,188,0,Y,2,0,2,0
2,1,1,0,30.0,1,N,28,71,1,0,1,215,58,N,0,0,1,0
3,1,3,0,23.7,2,N,23,64,1,1,1,138,0,N,0,0,2,0
4,3,1,0,35.5,1,N,37,66,1,1,1,220,0,N,0,0,1,4


In [10]:
# renaming columns
new_cols =['Delivery_Method','Medical_Attendant','Cigarette_Consumption','BMI','Marital_Status','Induction_of_labor'
               ,'Age_of_Mother','Mothers_Height_in_Inches','NO_INFEC','NO_RISK','NO_MORBIDITY','Pre_Pregnancy_Weight'
               ,'Weight_Gain','Previous_Cesarian','#_of_Previous_Cesareans','Prior_Births_Dead','Prior_Births_Living'
               ,'Prior_Terminations']

delivery_method.columns = new_cols

delivery_method.head()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Marital_Status,Induction_of_labor,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,Previous_Cesarian,#_of_Previous_Cesareans,Prior_Births_Dead,Prior_Births_Living,Prior_Terminations
0,1,1,0,30.7,1,N,30,66,1,1,1,190,41,N,0,0,1,2
1,3,1,2,33.3,2,N,35,63,1,0,1,188,0,Y,2,0,2,0
2,1,1,0,30.0,1,N,28,71,1,0,1,215,58,N,0,0,1,0
3,1,3,0,23.7,2,N,23,64,1,1,1,138,0,N,0,0,2,0
4,3,1,0,35.5,1,N,37,66,1,1,1,220,0,N,0,0,1,4


In [11]:
# Reduce the dataset size by selecting random 100000 rows to work with
delivery_method_df = delivery_method.sample(n=100000)
#save to excel
delivery_method_df.to_csv('delivery_prediction.csv')
delivery_method_df.head()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Marital_Status,Induction_of_labor,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,Previous_Cesarian,#_of_Previous_Cesareans,Prior_Births_Dead,Prior_Births_Living,Prior_Terminations
2770515,1,1,3,99.9,2,N,31,64,1,1,1,999,99,N,0,0,1,0
833426,3,1,0,23.4,1,N,32,66,1,0,1,145,22,N,0,0,1,0
2132254,1,1,0,20.0,1,N,34,63,1,1,1,113,20,N,0,0,2,0
1948736,3,1,0,22.8,1,N,35,68,1,1,1,150,30,N,0,0,0,0
2603390,1,1,0,32.4,1,Y,20,63,1,1,1,183,10,N,0,0,0,0


In [12]:
delivery_method_df.dtypes

Delivery_Method               int64
Medical_Attendant             int64
Cigarette_Consumption         int64
BMI                         float64
Marital_Status               object
Induction_of_labor           object
Age_of_Mother                 int64
Mothers_Height_in_Inches      int64
NO_INFEC                      int64
NO_RISK                       int64
NO_MORBIDITY                  int64
Pre_Pregnancy_Weight          int64
Weight_Gain                   int64
Previous_Cesarian            object
#_of_Previous_Cesareans       int64
Prior_Births_Dead             int64
Prior_Births_Living           int64
Prior_Terminations            int64
dtype: object

In [13]:
#used get_dummies to Convert categorical variable into dummy/indicator variables.
delivery_method_df=pd.get_dummies(delivery_method_df)
delivery_method_df.head()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,...,Marital_Status_2,Marital_Status_,Marital_Status_1,Marital_Status_2,Induction_of_labor_N,Induction_of_labor_U,Induction_of_labor_Y,Previous_Cesarian_N,Previous_Cesarian_U,Previous_Cesarian_Y
2770515,1,1,3,99.9,31,64,1,1,1,999,...,1,0,0,0,1,0,0,1,0,0
833426,3,1,0,23.4,32,66,1,0,1,145,...,0,0,0,0,1,0,0,1,0,0
2132254,1,1,0,20.0,34,63,1,1,1,113,...,0,0,0,0,1,0,0,1,0,0
1948736,3,1,0,22.8,35,68,1,1,1,150,...,0,0,0,0,1,0,0,1,0,0
2603390,1,1,0,32.4,20,63,1,1,1,183,...,0,0,0,0,0,0,1,1,0,0


### Split data into features and labels/Targets

In [14]:
# define target Column
target=delivery_method_df['Delivery_Method']
target_names=["vaginal","c-section"]


In [127]:
# Remove the labels from the features
# axis 1 refers to the columns
data=delivery_method_df.drop('Delivery_Method',axis=1)
# Saving feature names for later use
features_names=data.columns
data.head()

,Medical_Attendant,Cigarette_Consumption,BMI,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,...,Marital_Status_2,Marital_Status_,Marital_Status_1,Marital_Status_2,Induction_of_labor_N,Induction_of_labor_U,Induction_of_labor_Y,Previous_Cesarian_N,Previous_Cesarian_U,Previous_Cesarian_Y
2770515,1,3,99.9,31,64,1,1,1,999,99,...,1,0,0,0,1,0,0,1,0,0
833426,1,0,23.4,32,66,1,0,1,145,22,...,0,0,0,0,1,0,0,1,0,0
2132254,1,0,20.0,34,63,1,1,1,113,20,...,0,0,0,0,1,0,0,1,0,0
1948736,1,0,22.8,35,68,1,1,1,150,30,...,0,0,0,0,1,0,0,1,0,0
2603390,1,0,32.4,20,63,1,1,1,183,10,...,0,0,0,0,0,0,1,1,0,0


In [34]:
data.columns

Index(['Medical_Attendant', 'Cigarette_Consumption', 'BMI', 'Age_of_Mother',
       'Mothers_Height_in_Inches', 'NO_INFEC', 'NO_RISK', 'NO_MORBIDITY',
       'Pre_Pregnancy_Weight', 'Weight_Gain', '#_of_Previous_Cesareans',
       'Prior_Births_Dead', 'Prior_Births_Living', 'Prior_Terminations',
       'Marital_Status_1', 'Marital_Status_2', 'Marital_Status_ ',
       'Marital_Status_1', 'Marital_Status_2', 'Induction_of_labor_N',
       'Induction_of_labor_U', 'Induction_of_labor_Y', 'Previous_Cesarian_N',
       'Previous_Cesarian_U', 'Previous_Cesarian_Y'],
      dtype='object')

### Training and Testing Different Algorithms Sets

In [70]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data,target,random_state = 3)

In [71]:
# Verify the training model testing and traning features.We expect the training features number of columns to match the testing feature number of columns
# and the number of rows to match for the respective training and testing features and the labels
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (75000, 25)
Training Labels Shape: (75000,)
Testing Features Shape: (25000, 25)
Testing Labels Shape: (25000,)


### Selecting the model. Run Score tests for different algorithms

In [77]:
#Import Algorithm to use
from sklearn.ensemble import RandomForestClassifier
#Create RF classifier
rf = RandomForestClassifier(n_estimators=200,max_depth=7)
# Train random forest Classifier
rf = rf.fit(X_train,y_train)
# # Get the scores
# rf.score(X_test,y_test)
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 0.8037066666666667
Testing Data Score: 0.80516


In [78]:
from sklearn.linear_model import LogisticRegression
#Create Logistic regression
cls = LogisticRegression()
# Train logistic regression Classifier
cls = cls.fit(X_train,y_train)
# # Get the scores
# cls.score(X_test,y_test)

print(f"Training Data Score: {cls.score(X_train, y_train)}")
print(f"Testing Data Score: {cls.score(X_test, y_test)}")

C:\Users\Mercy\Anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Training Data Score: 0.7990933333333333
Testing Data Score: 0.80216


We perfomed 2 classifier tests on Random forest, logistic regression and Random forest had a slighter better out put.some of the advantages of Random forest are 
 higher accuracy, handles the missing values and maintains the accuracy of a large proportion of data. If there are more trees, it won’t allow overfitting trees in the model. It has the power to handle a large data set with higher dimensionality

In [128]:
# In order to quantify the usefulness of all the variables in the entire random forest
importances = list(rf.feature_importances_)
sorted(zip(rf.feature_importances_,features_names),reverse=True)

[(0.2410703991687634, 'Previous_Cesarian_Y'),
 (0.22258969680720542, 'Previous_Cesarian_N'),
 (0.16310959689718985, '#_of_Previous_Cesareans'),
 (0.11831909296937385, 'NO_RISK'),
 (0.08038617096850739, 'Medical_Attendant'),
 (0.06264723649052677, 'Prior_Births_Living'),
 (0.02839894505048667, 'Induction_of_labor_Y'),
 (0.022308780330370096, 'Induction_of_labor_N'),
 (0.019849723349709227, 'BMI'),
 (0.012786868677606733, 'Age_of_Mother'),
 (0.012322598235589418, 'Pre_Pregnancy_Weight'),
 (0.006256231252895215, 'Weight_Gain'),
 (0.004179827246556718, 'Mothers_Height_in_Inches'),
 (0.0013746192946662735, 'Prior_Terminations'),
 (0.000980310725749254, 'NO_MORBIDITY'),
 (0.00074192364744967, 'Cigarette_Consumption'),
 (0.0005323793149220445, 'Marital_Status_1'),
 (0.00046641093515133956, 'Previous_Cesarian_U'),
 (0.0004639233696575325, 'Marital_Status_ '),
 (0.0004395323481473978, 'Marital_Status_2'),
 (0.0002703745903234603, 'NO_INFEC'),
 (0.0002578857194380808, 'Prior_Births_Dead'),
 (0.0

### Make Predictions on the Test Set

Confusion Matrix in Machine Learning
In the field of machine learning and specifically the problem of statistical classification, a confusion matrix, also known as an error matrix. A confusion matrix is a table that is often used to describe the performance of a classification model (or “classifier”) on a set of test data for which the true values are known.
It allows the visualization of the performance of an algorithm.

In [92]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predictions)

array([[16597,   419],
       [ 4452,  3532]], dtype=int64)

In [ ]:
from sklearn.metrics import confusion_matrix

In [48]:
y_test.iloc[1:]

685763     3
2421772    3
3743512    1
1612092    1
3085932    3
          ..
800145     1
2794515    1
74550      3
2041992    1
3147984    1
Name: Delivery_Method, Length: 24999, dtype: int64

### Improve Model if Necessary

#### Create samples from unbiased data for each target

In [134]:
# # create two datafames to provide equal samples to be used to train the model
vaginal_df = delivery_method_df[delivery_method_df.Delivery_Method == 1]
#create a random sample of 30000 vaginal records
vaginal_df = vaginal_df.sample(n=30000)
csection_df = delivery_method_df[delivery_method_df.Delivery_Method == 3]
#create a random sample of 30000 section records
csection_df = csection_df.sample(n=30000)
#combine the two dataframes to a final one
delivery_method_df2=pd.concat([vaginal_df, csection_df])
# define target Column
target1=delivery_method_df2['Delivery_Method']
target_names1=["vaginal","c-section"]
# Remove the labels from the features and less than 5% importance
# axis 1 refers to the columns
data1=delivery_method_df2.drop('Delivery_Method',axis=1)
# Saving feature names for later use
features_names1=data1.columns
data1.shape


(60000, 25)

In [95]:
#Import Algorithm to use
from sklearn.ensemble import RandomForestClassifier
X1_train,X1_test,y1_train,y1_test = train_test_split(data1,target1,random_state = 3)

In [125]:
#Create RF classifier
rf1 = RandomForestClassifier(n_estimators=200,max_depth=7)
# Train random forest Classifier
rf1 = rf1.fit(X1_train,y1_train)
# # Get the scores
# rf.score(X_test,y_test)
# print(f"Training Data Score: {rf.score(X_train, y_train)}")
# print(f"Testing Data Score: {rf.score(X_test, y_test)}")
print(f"Training Data Score: {rf1.score(X1_train, y1_train)}")
print(f"Testing Data Score: {rf1.score(X1_test, y1_test)}")

Training Data Score: 0.7489777777777777
Testing Data Score: 0.7452666666666666


##### remove columns of less importance

In [182]:
# define target Column
target2=delivery_method_df2['Delivery_Method']
target_names2=["vaginal","c-section"]
# Remove the labels from the features and less than 5% importance
# axis 1 refers to the columns
data2=delivery_method_df2.drop(['Delivery_Method',  \
#                                 'Medical_Attendant','Prior_Births_Living', 'Induction_of_labor_Y',  'Induction_of_labor_N',  'BMI',  'Age_of_Mother','Pre_Pregnancy_Weight',  'Weight_Gain','Mothers_Height_in_Inches',\
#                                 'Prior_Terminations','NO_MORBIDITY','Cigarette_Consumption',  'Marital_Status_1','Previous_Cesarian_U','Marital_Status_ ','Marital_Status_2',   'NO_INFEC',  
                           'Prior_Births_Dead',   'Marital_Status_1','Induction_of_labor_U',  'Marital_Status_2'],axis=1)
# 'Medical_Attendant', 'NO_RISK',  'Prior_Terminations', 'Cigarette_Consumption',, 
#                                ],axis=1)
# Saving feature names for later use
features_names2=data2.columns
data2.shape


(60000, 19)

In [161]:
# In order to quantify the usefulness of all the variables in the entire random forest
importances = list(rf.feature_importances_)
sorted(zip(rf.feature_importances_,features_names),reverse=True)

[(0.2410703991687634, 'Previous_Cesarian_Y'),
 (0.22258969680720542, 'Previous_Cesarian_N'),
 (0.16310959689718985, '#_of_Previous_Cesareans'),
 (0.11831909296937385, 'NO_RISK'),
 (0.08038617096850739, 'Medical_Attendant'),
 (0.06264723649052677, 'Prior_Births_Living'),
 (0.02839894505048667, 'Induction_of_labor_Y'),
 (0.022308780330370096, 'Induction_of_labor_N'),
 (0.019849723349709227, 'BMI'),
 (0.012786868677606733, 'Age_of_Mother'),
 (0.012322598235589418, 'Pre_Pregnancy_Weight'),
 (0.006256231252895215, 'Weight_Gain'),
 (0.004179827246556718, 'Mothers_Height_in_Inches'),
 (0.0013746192946662735, 'Prior_Terminations'),
 (0.000980310725749254, 'NO_MORBIDITY'),
 (0.00074192364744967, 'Cigarette_Consumption'),
 (0.0005323793149220445, 'Marital_Status_1'),
 (0.00046641093515133956, 'Previous_Cesarian_U'),
 (0.0004639233696575325, 'Marital_Status_ '),
 (0.0004395323481473978, 'Marital_Status_2'),
 (0.0002703745903234603, 'NO_INFEC'),
 (0.0002578857194380808, 'Prior_Births_Dead'),
 (0.0

In [188]:
#Import Algorithm to use
from sklearn.ensemble import RandomForestClassifier
X2_train,X2_test,y2_train,y2_test = train_test_split(data2,target2,random_state = 3)

In [184]:
#Create RF classifier
rf2 = RandomForestClassifier(n_estimators=200,max_depth=7)
# Train random forest Classifier
rf2 = rf2.fit(X2_train,y2_train)
# # Get the scores
# rf.score(X_test,y_test)
# print(f"Training Data Score: {rf.score(X_train, y_train)}")
# print(f"Testing Data Score: {rf.score(X_test, y_test)}")
print(f"Training Data Score: {rf2.score(X2_train, y2_train)}")
print(f"Testing Data Score: {rf2.score(X2_test, y2_test)}")

Training Data Score: 0.7522444444444445
Testing Data Score: 0.7476


##### Adjust hyperparameters on unbiased data

In [189]:
#Import Algorithm to use
from sklearn.ensemble import RandomForestClassifier
X3_train,X3_test,y3_train,y3_test = train_test_split(data2,target2,random_state = 3)

In [199]:
#Create RF classifier
rf3 = RandomForestClassifier(n_estimators=300,max_depth=6)
# Train random forest Classifier
rf3 = rf3.fit(X3_train,y3_train)
# # Get the scores
# rf.score(X_test,y_test)
# print(f"Training Data Score: {rf.score(X_train, y_train)}")
# print(f"Testing Data Score: {rf.score(X_test, y_test)}")
print(f"Training Data Score: {rf3.score(X3_train, y3_train)}")
print(f"Testing Data Score: {rf3.score(X3_test, y3_test)}")

Training Data Score: 0.7449111111111111
Testing Data Score: 0.744


### Train the model and run scores for random Forest on the new data set

In [152]:
#Import Algorithm to use
from sklearn.ensemble import RandomForestClassifier
X2_train,X2_test,y2_train,y2_test = train_test_split(data2,target2,random_state = 3)

In [153]:
#Create RF classifier
rf2 = RandomForestClassifier(n_estimators=200,max_depth=7)
# Train random forest Classifier
rf2 = rf2.fit(X2_train,y2_train)
# # Get the scores
# rf.score(X_test,y_test)
# print(f"Training Data Score: {rf.score(X_train, y_train)}")
# print(f"Testing Data Score: {rf.score(X_test, y_test)}")
print(f"Training Data Score: {rf2.score(X2_train, y2_train)}")
print(f"Testing Data Score: {rf2.score(X2_test, y2_test)}")

Training Data Score: 0.6987777777777778
Testing Data Score: 0.6964666666666667


In [ ]:
# Use the forest's predict method on the test data
predictions2 = rf2.predict(X2_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y2_test, predictions2)

### Interpret Model and Report Results

### Visualizing a Single Decision Tree

In [ ]:
# One of the coolest parts of the Random Forest implementation in Skicit-learn is we can actually 
# examine any of the trees in the forest. 
# We will select one tree, and save the whole tree as an image
# The following code takes one tree from the forest and saves it as an image

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

In [ ]:
# Pull out one tree from the forest
tree = rf2.estimators_[4]

In [ ]:
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_names2, rounded = True, precision = 1)

In [ ]:
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

In [ ]:
# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestClassifier(n_estimators=10, max_depth = 3)
rf_small.fit(X2_train, y2_train)

In [ ]:
# Extract the small tree
tree_small = rf_small.estimators_[5]

In [ ]:
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_names2, rounded = True, precision = 1)

In [ ]:
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [ ]:
### get only numeric features  by selecting only features that are not "object" datatypes
# numeric_variables=list(delivery_method_df.dtypes[delivery_method_df.dtypes != "object"].index)
# delivery_method_df[numeric_variables].head()

In [ ]:
#build first model
# model=RandomForestRegressor(n_estimators=100,oob_score=True, random_state=42)
# Fit the model
# model.fit(delivery_method_df[numeric_variables,Y ]


### Visualization

In [ ]:
# a simple bar plot of the feature importances 
# to illustrate the disparities in the relative significance of the variables
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')

# list of x locations for plotting
x_values = list(range(len(importances)))



In [ ]:
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# plot the entire dataset with predictions highlighted
# Use datetime for creating date objects for plotting
import datetime
# Dates of training values
months = features[:, feature_list.index('month')]
days = features[:, feature_list.index('day')]
years = features[:, feature_list.index('year')]
# List and then convert to datetime object
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]
# Dataframe with true values and dates
true_data = pd.DataFrame(data = {'date': dates, 'actual': labels})
# Dates of predictions
months = test_features[:, feature_list.index('month')]
days = test_features[:, feature_list.index('day')]
years = test_features[:, feature_list.index('year')]
# Column of dates
test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
# Convert to datetime objects
test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]
# Dataframe with predictions and dates
predictions_data = pd.DataFrame(data = {'date': test_dates, 'prediction': predictions})
# Plot the actual values
plt.plot(true_data['date'], true_data['actual'], 'b-', label = 'actual')
# Plot the predicted values
plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'prediction')
plt.xticks(rotation = '60'); 
plt.legend()
# Graph labels
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual and Predicted Values');


In [ ]:
# Make the data accessible for plotting
true_data['temp_1'] = features[:, feature_list.index('temp_1')]
true_data['average'] = features[:, feature_list.index('average')]
true_data['friend'] = features[:, feature_list.index('friend')]
# Plot all the data as lines
plt.plot(true_data['date'], true_data['actual'], 'b-', label  = 'actual', alpha = 1.0)
plt.plot(true_data['date'], true_data['temp_1'], 'y-', label  = 'temp_1', alpha = 1.0)
plt.plot(true_data['date'], true_data['average'], 'k-', label = 'average', alpha = 0.8)
plt.plot(true_data['date'], true_data['friend'], 'r-', label = 'friend', alpha = 0.3)
# Formatting plot
plt.legend(); plt.xticks(rotation = '60');
# Lables and title
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual Max Temp and Variables');